## Refactoring play by play
They made some updates to nflfastr that changes the schema

In [ ]:
import logging
import os
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from app import db

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows', 50)

db_conn = db.get_db_conn()

In [29]:
REMOTE_PATH_TEMPLATE = 'https://github.com/guga31bb/nflfastR-data/blob/master/data/play_by_play_2020.csv.gz?raw=True'

def _extract(path: str) -> pd.DataFrame:
    """Download CSV from remote path."""
    logging.info(f"Downloading remote play by play CSV from {path}...")
    return pd.read_csv(
        path,
        compression='gzip' if "csv.gz" in path else "infer",
        low_memory=False
    )

year = 2020
df = _extract(REMOTE_PATH_TEMPLATE.format(year=year))
df['year'] = year

In [30]:
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [41]:
df.to_csv("new_pbp.csv", index=False)

In [32]:
df_old = pd.read_sql("""SELECT * FROM play_by_play""", db_conn)
df_old.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [33]:
old_cols = set(df_old.columns)
new_cols = set(df.columns)

In [34]:
added_cols = new_cols - old_cols
missing_cols = old_cols - new_cols
added_cols, missing_cols

(set(), set())

In [39]:
mask = (df['surface'].isna())
test = df[mask]
test['game_id'].value_counts()

2020_21_KC_TB    184
Name: game_id, dtype: int64

In [37]:
df['surface'].value_counts()

grass         28204
fieldturf     7581 
astroturf     5348 
matrixturf    2880 
fieldturf     2835 
sportturf     1482 
Name: surface, dtype: int64

In [38]:
sum(df['surface'].isna())

184

In [40]:
db_conn.close()

In [52]:
REMOTE_PATH = 'https://raw.githubusercontent.com/mrcaseb/nflfastR-roster/master/data/seasons/roster_{year}.csv'
def _extract(path: str) -> pd.DataFrame:
    """Download CSV from remote path."""
    logging.info(f"Downloading remote play by play CSV from {path}...")
    return pd.read_csv(
        path,
        compression='gzip' if "csv.gz" in path else "infer",
        low_memory=False
    )

year = 2020
df = _extract(REMOTE_PATH.format(year=year))


In [53]:
df.head()

,season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,weight,college,high_school,gsis_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,fantasy_data_id,sleeper_id,update_dt,headshot_url
0,2020,ARI,C,C,52.0,Active,Mason Cole,Mason,Cole,1996-03-28,6-5,292.0,Michigan,East Lake (FL),00-0034785,3115972.0,53d25371-e3ce-4030-8d0a-82def5cdc600,31067.0,12795.0,45547.0,19898.0,5062,2021-02-21T07:10:22Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/3115972.png
1,2020,ARI,C,C,53.0,Active,Lamont Gaillard,Lamont,Gaillard,1996-02-08,6-3,305.0,Georgia,Pine Forest (NC),00-0035536,3128707.0,0c8b0581-9ed2-488b-bcaa-ef783261dfd1,32011.0,13678.0,46618.0,20763.0,6058,2021-02-21T07:10:22Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/3128707.png
2,2020,ARI,CB,NB,33.0,Active,Byron Murphy,Byron,Murphy,1998-01-18,5-11,190.0,Washington,Saguaro (AZ),00-0035236,4038999.0,c025b513-9431-4097-bc25-9777bf08f846,31865.0,13560.0,44027.0,20888.0,5864,2021-02-21T07:10:22Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/4038999.png
3,2020,ARI,CB,NaN,20.0,Active,Prince Amukamara,Prince,Amukamara,1989-06-06,6-0,204.0,Nebraska,Apollo (AZ),00-0027957,13975.0,f1879cfa-4c07-4140-9da0-c7ebe9af2dfd,24806.0,7509.0,6171.0,13135.0,908,2021-02-21T07:10:22Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/13975.png
4,2020,ARI,CB,NaN,39.0,Active,Jace Whittaker,Jace,Whittaker,1995-07-16,5-11,185.0,Arizona,Oceanside (CA),00-0036043,3821572.0,528bf5c9-1d23-40c3-adda-df21f8f0e2ab,33089.0,14793.0,50465.0,22171.0,7169,2021-02-21T07:10:22Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/3821572.png


In [54]:
df_old = pd.read_sql("""SELECT * FROM roster""", db_conn)
df_old.head()

,season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,weight,college,high_school,gsis_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,update_dt,headshot_url


In [55]:
old_cols = set(df_old.columns)
new_cols = set(df.columns)
added_cols = new_cols - old_cols
missing_cols = old_cols - new_cols
added_cols, missing_cols

({'fantasy_data_id', 'sleeper_id'}, set())

In [1]:
db_conn.close()

NameError: name 'db_conn' is not defined

## Having trouble joining roster to pbp

In [6]:
df_pbp = pd.read_sql("""SELECT * FROM play_by_play_enriched WHERE game_id = '2020_01_ARI_SF'""", db_conn)
df_roster = pd.read_sql("""SELECT * FROM roster""", db_conn)

In [10]:
test = df_pbp[(~df_pbp['receiver_id'].isna()) & (df_pbp['receiver_position'].isna())]
test[['desc', 'receiver', 'receiver_position', 'receiver_id', 'receiver_gsis_id']].head(20)

,desc,receiver,receiver_position,receiver_id,receiver_gsis_id
2,"(15:00) (Shotgun) 10-J.Garoppolo pass short right to 85-G.Kittle to SF 30 for 5 yards (48-I.Simmons). PENALTY on ARI-48-I.Simmons, Horse Collar Tackle, 15 yards, enforced at SF 30. Caught at SF29. 1-YAC",G.Kittle,None,00-0033288,
6,"(12:39) (Shotgun) 10-J.Garoppolo pass short middle to 85-G.Kittle to ARI 34 for 11 yards (58-J.Hicks). Caught at AZ 48, 14-YAC",G.Kittle,None,00-0033288,
9,(11:50) (Shotgun) 1-K.Murray pass short left to 10-D.Hopkins to ARI 28 for 3 yards (41-E.Moseley).,D.Hopkins,None,00-0030564,
10,"(11:24) (No Huddle, Shotgun) 1-K.Murray pass short left to 11-L.Fitzgerald to ARI 35 for 7 yards (41-E.Moseley; 20-J.Ward). caught at AZ 34, 1 YAC",L.Fitzgerald,None,00-0022921,
12,(10:58) (Shotgun) 1-K.Murray pass incomplete short right to 11-L.Fitzgerald (54-F.Warner).,L.Fitzgerald,None,00-0022921,
13,"(10:55) (No Huddle, Shotgun) 1-K.Murray pass short right to 11-L.Fitzgerald to ARI 35 for 10 yards (29-J.Tartt). caught at 25R, 10 YAC",L.Fitzgerald,None,00-0022921,
15,"(10:05) (Shotgun) 10-J.Garoppolo pass short middle to 31-R.Mostert for 76 yards, TOUCHDOWN. Caught at SF 29, 71-YAC",R.Mostert,None,00-0031687,
18,"(9:53) (Shotgun) 1-K.Murray pass short left to 10-D.Hopkins to ARI 26 for 1 yard (25-R.Sherman). caught at ARI 21, 5 YAC",D.Hopkins,None,00-0030564,
19,"(9:20) (No Huddle, Shotgun) 1-K.Murray pass short right to 41-K.Drake to ARI 28 for 2 yards (91-A.Armstead; 54-F.Warner). caught at ARI 20, 8 YAC",K.Drake,None,00-0033118,
20,"(8:43) (No Huddle, Shotgun) 1-K.Murray pass incomplete deep right to 13-C.Kirk.",C.Kirk,None,00-0034775,


In [61]:
df_roster[df_roster['full_name'] == 'George Kittle']

,season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,weight,college,high_school,gsis_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,update_dt,headshot_url,fantasy_data_id,sleeper_id
2687,2020,SF,TE,LTE,85.0,Active,George Kittle,George,Kittle,1993-10-09,6-4,250.0,Iowa,Norman (OK),00-0033288,3040151.0,2ada91b0-036e-454f-83c3-6d939ff584a9,30259.0,11892.0,11901.0,2021-02-21 07:10:22,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/3040151.png,19063.0,4217


In [62]:
df_roster_slim = df_roster[['season', 'position', 'gsis_id']]
df_roster_slim = df_roster_slim.drop_duplicates()
df_roster_slim = df_roster_slim[df_roster_slim['position'].notnull()]
df_roster_slim = df_roster_slim[df_roster_slim['gsis_id'].notnull()]

In [64]:
df_enriched = df_pbp.merge(
    df_roster_slim,
    how='left',
    left_on=['season', 'receiver_id'],
    right_on=['season', 'gsis_id']
)

In [69]:
test = df_enriched[df_enriched['receiver'] == 'G.Kittle'][['receiver', 'receiver_id', 'position']]

In [71]:
test['position'].isna().sum()

0